## MNIST Classification using PyTorch
Implement a CNN model in PyTorch to classify MNIST dataset, you can use sklearn library to import the mnist dataset.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# hyper parameters
num_epochs = 4
batch_size = 100
learning_rate = 0.01

In [4]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [5]:
# MNIST Data
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset=  torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

In [6]:
class Convnet(nn.Module):
    def __init__(self):
        super(Convnet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5) 
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1_input_size = 16*4*4
        self.fc1 = nn.Linear(self.fc1_input_size, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, self.fc1_input_size)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
model = Convnet().to(device)

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [8]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 200 == 0:
            print(f'epoch {epoch+1}/{num_epochs}, Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')

epoch 1/4, Step [200/600], Loss: 2.2905
epoch 1/4, Step [400/600], Loss: 2.2863
epoch 1/4, Step [600/600], Loss: 2.2663
epoch 2/4, Step [200/600], Loss: 2.1641
epoch 2/4, Step [400/600], Loss: 1.2615
epoch 2/4, Step [600/600], Loss: 0.6160
epoch 3/4, Step [200/600], Loss: 0.6246
epoch 3/4, Step [400/600], Loss: 0.3840
epoch 3/4, Step [600/600], Loss: 0.3417
epoch 4/4, Step [200/600], Loss: 0.1942
epoch 4/4, Step [400/600], Loss: 0.2362
epoch 4/4, Step [600/600], Loss: 0.1915
Finished Training


In [9]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        n_samples += (labels.size(0))
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc}%')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc}%')

Accuracy of the network: 92.87%
Accuracy of 0: 98.87755102040816%
Accuracy of 1: 97.88546255506608%
Accuracy of 2: 93.02325581395348%
Accuracy of 3: 92.97029702970298%
Accuracy of 4: 91.44602851323829%
Accuracy of 5: 92.37668161434978%
Accuracy of 6: 94.67640918580376%
Accuracy of 7: 91.53696498054475%
Accuracy of 8: 86.55030800821355%
Accuracy of 9: 88.70168483647176%
